In [9]:
import pandas as pd
import numpy as np
import json
import re
from urllib.parse import urlparse


In [38]:
from datetime import datetime

In [39]:
# Load JSON data
input_file = 'Aec_data.json'
output_file = 'cleaned_Aec_data.json'

In [40]:
# Assuming the provided data is saved as aec_news_data.json
with open(input_file, 'r', encoding='utf-8') as f:
    data = json.load(f)


In [41]:
# Convert to DataFrame
df = pd.DataFrame(data)

In [42]:
# Step 1: Handle missing values
df['description'] = df['description'].fillna('')
df['content'] = df['content'].fillna('')
df['author'] = df['author'].fillna('unknown')
df['url_to_image'] = df['url_to_image'].fillna('none')
df = df.dropna(subset=['title', 'query_category'])

In [43]:
# Step 2: Clean text fields
def clean_text(text):
    if pd.isna(text):
        return ''
    # Remove URLs
    text = re.sub(r'http[s]?://\S+', '', text)
    # Remove HTML/Markdown tags (e.g., <ul>, <li>)
    text = re.sub(r'<[^>]+>', '', text)
    # Remove special characters (including *, [], Unicode \u2026, punctuation, emojis)
    text = re.sub(r'[^\w\s]', '', text)
    # Remove extra Unicode artifacts
    text = text.encode('ascii', 'ignore').decode('ascii')
    # Normalize whitespace, remove newlines
    text = ' '.join(text.split())
    return text

In [44]:
# Apply cleaning to title, description, and content
df['title'] = df['title'].apply(clean_text)
df['description'] = df['description'].apply(clean_text)
df['content'] = df['content'].apply(clean_text)

In [45]:
# Step 3: Clean author field
def clean_author(author):
    # Remove URLs and extra metadata
    author = re.sub(r'http[s]?://\S+', '', author)
    # Remove Contributor or similar suffixes
    author = re.sub(r', Contributor.*$', '', author, flags=re.IGNORECASE)
    # Split multiple authors, keep names only
    authors = [name.strip() for name in author.split(',') if name.strip()]
    return ', '.join(authors).lower()

df['author'] = df['author'].apply(clean_author)

In [46]:
# Step 4: Add text length
df['text_length'] = df['content'].str.len()

In [47]:
# Step 5: Convert dates to datetime
df['published_at'] = pd.to_datetime(df['published_at'], errors='coerce')
df['collection_date'] = pd.to_datetime(df['collection_date'], errors='coerce')

In [48]:
# Step 6: Categorize URLs
def categorize_url(url):
    if pd.isna(url):
        return 'none'
    parsed = urlparse(url)
    domain = parsed.netloc.lower()
    if any(x in domain for x in ['globenewswire', 'forbes', 'financialpost']):
        return 'news'
    elif any(x in domain for x in ['blog', 'lifeofanarchitect', 'christopherspenn']):
        return 'blog'
    return 'other'

df['url_type'] = df['url'].apply(categorize_url)

In [49]:
# Step 7: Normalize query_category and source
df['query_category'] = df['query_category'].str.lower().str.replace(',', '')
df['source'] = df['source'].str.lower()



In [50]:

# Step 8: Remove duplicates
df = df.drop_duplicates(subset=['title', 'url', 'query_category'])

In [51]:
df.head(3)

,title,description,content,url,source,published_at,author,url_to_image,query_category,query_used,collection_date,date_range,text_length,url_type
0,egnyte introduces enhanced partner program and...,mountain view calif may 20 2025 globe newswire...,mountain view calif may 20 2025 globe newswire...,https://www.globenewswire.com/news-release/202...,globenewswire,2025-05-20 15:00:00+00:00,"egnyte, inc",https://ml.globenewswire.com/Resource/Download...,industry_general,AEC industry OR Architecture Engineering Const...,2025-05-26 23:21:02.156361,2025-05-20_to_2025-05-23,184,news
1,intel arc pro bseries gpus with xmx ai cores a...,intel arc pro bseries graphics processing unit...,intel arc pro bseries graphics processing unit...,https://www.gadgets360.com/laptops/news/intel-...,gadgets360.com,2025-05-20 12:18:10+00:00,"shaurya tomer, siddharth suvarna",https://i.gadgets360cdn.com/large/intel_arc_pr...,industry_general,AEC industry OR Architecture Engineering Const...,2025-05-26 23:21:02.156361,2025-05-20_to_2025-05-23,194,other
2,the invisible technology behind the worlds mos...,gis powered digital twins enable modern aec fi...,modern engineering and construction firms are ...,https://www.forbes.com/sites/esri/2025/05/20/t...,forbes,2025-05-20 12:00:00+00:00,"kathleen kewley, contributor, kathleen kewley,...",https://imageio.forbes.com/specials-images/ima...,industry_general,AEC industry OR Architecture Engineering Const...,2025-05-26 23:21:02.156361,2025-05-20_to_2025-05-23,194,news


In [52]:
# Step 9: Save cleaned data
cleaned_data = df.to_dict(orient='records')
with open(output_file, 'w', encoding='utf-8') as f:
    json.dump(cleaned_data, f, ensure_ascii=False, indent=4, default=str)

print(f'Successfully saved cleaned data to {output_file}')

Successfully saved cleaned data to cleaned_Aec_data.json
